In [108]:
# Importing libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [109]:
print(os.listdir('BRS'))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'BRS'

In [110]:
print(os.listdir())

['app.py', 'archive (1)', 'Books.csv', 'model', 'notebook.ipynb', 'Ratings.csv', 'tempCodeRunnerFile.py', 'Templates', 'Users.csv']


In [111]:
books = pd.read_csv('Books.csv',on_bad_lines="skip")


In [112]:
#Explorinf the dataframe
books


NameError: name 'Image' is not defined

In [15]:
print(f"In thos DataFrame, there are:\n{books.shape[0]} rows,\n{books.shape[1]} colums")

In thos DataFrame, there are:
271360 rows,
8 colums


In [114]:
books.head()



'http://images.amazon.com/images/P/0671870432.01.MZZZZZZZ.jpg'

In [17]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [18]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [19]:
books.dtypes

ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object

In [20]:
books.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271358,271360,271358,271360,271360,271357
unique,271360,242135,102022,202,16807,271044,271044,271041
top,020130998X,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/155936078X.0...,http://images.amazon.com/images/P/155936078X.0...,http://images.amazon.com/images/P/155936078X.0...
freq,1,27,632,13903,7535,2,2,2


In [21]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [22]:
numeric_books = books.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numeric_books.corr()
print(correlation_matrix)

Empty DataFrame
Columns: []
Index: []


#### Popularity Based Recommendation System

In [23]:
ratings= pd.read_csv('ratings.csv')
users= pd.read_csv('users.csv')

In [24]:
books_rating = ratings.merge(books, on= 'ISBN')

In [25]:
# Grouping by 'Book-Title' and counting ratings for each book
book_rating_count_df = books_rating.groupby('Book-Title').count()['Book-Rating'].reset_index()

# Renaming the column to 'num_ratings'
book_rating_count_df.rename(columns={"Book-Rating": "book_rating_count"}, inplace=True)

book_rating_count_df

,Book-Title,book_rating_count
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [118]:
# Convert 'Book-Rating' to numeric, forcing errors to NaN (invalid data is replaced with NaN)
books_rating['Book-Rating'] = pd.to_numeric(books_rating['Book-Rating'], errors='coerce')

# Drop rows where 'Book-Rating' is NaN (optional, depending on your needs)
books_rating = books_rating.dropna(subset=['Book-Rating'])

# Grouping by 'Book-Title' and calculating the average rating for each book
book_avg_rating_df = books_rating.groupby('Book-Title')['Book-Rating'].mean().reset_index()

# Renaming the column to 'avg_ratings'
book_avg_rating_df.rename(columns={"Book-Rating": "avg_ratings"}, inplace=True)

# Rounding the 'avg_ratings' to 1 decimal place
book_avg_rating_df['avg_ratings'] = book_avg_rating_df['avg_ratings'].round(1)

book_avg_rating_df


,Book-Title,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,2.2
1,Always Have Popsicles,0.0
2,Apple Magic (The Collector's series),0.0
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.0
4,Beyond IBM: Leadership Marketing and Finance ...,0.0
...,...,...
241066,Ã?Â?lpiraten.,0.0
241067,Ã?Â?rger mit Produkt X. Roman.,5.2
241068,Ã?Â?sterlich leben.,7.0
241069,Ã?Â?stlich der Berge.,2.7


In [119]:
# Merging book_rating_count_df and book_avg_rating_df on 'Book-Title'
popular_df = book_rating_count_df.merge(book_avg_rating_df, on='Book-Title')
popular_df.head()



,Book-Title,book_rating_count,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.2
1,Always Have Popsicles,1,0.0
2,Apple Magic (The Collector's series),1,0.0
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.0
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.0


In [120]:
popular_df = popular_df[popular_df['book_rating_count'] > 250].sort_values('avg_ratings', ascending=False).head(50)
print(popular_df.shape)

(50, 3)


In [121]:
popular_df = popular_df.merge(books, on = 'Book-Title').drop_duplicates('Book-Title')[['Book-Title', 'Book-Author', 'Image-URL-M', 'book_rating_count','avg_ratings']]
popular_df

,Book-Title,Book-Author,Image-URL-M,book_rating_count,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.9
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.8
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.7
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.5
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.2
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.0
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.9
26,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.9
37,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.9
39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.7


### Collaborative Filtering Based Recoomendation System

In [122]:
df1 = books_rating.groupby('User-ID').count()['Book-Rating']>200
experinced_users= df1[df1].index

In [123]:
filtered_ratings = books_rating [books_rating['User-ID'].isin(experinced_users)
]

In [124]:
y= filtered_ratings.groupby('Book-Title').count()['Book-Rating']>=50
famous_book = y[y].index

In [125]:
final_ratings = filtered_ratings[filtered_ratings['Book-Title'].isin(famous_book)]

In [126]:
pt = final_ratings.pivot_table(index = 'Book-Title', columns='User-ID', values = 'Book-Rating')

In [127]:
pt.fillna(0, inplace = True)

In [128]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
from sklearn.metrics.pairwise import cosine_similarity

In [130]:
similarity_score = cosine_similarity(pt)

In [131]:
similarity_score

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [140]:
def recommend(book_name):
    #indexfetch
    index = np.where(pt.index== book_name )[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])), key= lambda x:x[1], reverse = True)[1:6]
 
    data=[]
    for i in  similar_items :
        item=[]
        temp_df =books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))


        data.append(item)

    return data




    


In [138]:
recommend('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg'],
 ['The Hours : A Novel',
  'Michael Cunningham',
  'http://images.amazon.com/images/P/0312243022.01.MZZZZZZZ.jpg']]

In [134]:
import pickle
pickle.dump(popular_df, open('popular.pkl', 'wb'))

# To verify, you can try loading it back
loaded_df = pickle.load(open('popular.pkl', 'rb'))
print(loaded_df)

                                            Book-Title  \
0    Harry Potter and the Prisoner of Azkaban (Book 3)   
3         Harry Potter and the Goblet of Fire (Book 4)   
5       Harry Potter and the Sorcerer's Stone (Book 1)   
9    Harry Potter and the Order of the Phoenix (Boo...   
13    Harry Potter and the Chamber of Secrets (Book 2)   
16   The Hobbit : The Enchanting Prelude to The Lor...   
17   The Fellowship of the Ring (The Lord of the Ri...   
26      The Two Towers (The Lord of the Rings, Part 2)   
37   Harry Potter and the Sorcerer's Stone (Harry P...   
39                               To Kill a Mockingbird   
47                  The Five People You Meet in Heaven   
49                                   The Da Vinci Code   
55                            Prodigal Summer: A Novel   
56                              The Catcher in the Rye   
63                                                1984   
72                           The Lovely Bones: A Novel   
73   Tuesdays 

In [139]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books, open('books.pkl', 'wb'))
pickle.dump(similarity_score, open('similarity_score.pkl', 'wb'))